# Solve the polynomial optimization problem using the moment-SOS hierarchy

In [2]:
using JSON
using TSSOS
using DynamicPolynomials

In [4]:
# Define variables
@polyvar x1 x2 x3

(x1, x2, x3)

In [5]:
## Extract polynomial to minimize from file
polynomial_mpo0 = JSON.parsefile("polynomial_mpo.json")

function parse_polynomial(p::String)
    expr = Meta.parse(p)
    return eval(expr)
end

poly_mpo = parse_polynomial(polynomial_mpo0)

4.93480220054468x1² + 3.87578458503748x1x2 + 2.02935606320838x1x3 + 0.813167225611526x2² + 0.878856959829747x2x3 + 0.240808917252909x3²

#### Code from Victor Magron for extraction of solutions via TSSOS

In [7]:
using LinearAlgebra

# extract independent columns of the matrix A
# r is the expected rank
function i_cols(A, r)
  QR = qr(A, Val(true))
    return QR.p[1:r]
end

# See https://homepages.laas.fr/henrion/papers/extract.pdf for a description of the extraction algorithm
# Md is the pseudo-moment matrix, assumed to be flat with rank r, returned by data.moment[1]
# vd is the monomial vector returned by data.basis[1] or data.basis in the unconstrained case
# s = size of the sub-moment matrix subM assumed to have the same rank as Md, e.g., M_{d-1}
# n is the number of variables

function extract(Md,vd,s,n)

subM=Md[1:s,1:s];
ic = sort(i_cols(subM,r));
subv = map(i->vd[:,i],ic);
C = Md[:,ic];
hatH = Md[ic,ic];
G = cholesky(hatH).U;

In=UInt8.(Matrix(I,n,n));

idx1 = map(m -> In[1,:]+m ,subv);
idx2 = map(m -> In[2,:]+m ,subv);
idx3 = map(m -> In[3,:]+m ,subv);

vdv = [vd[:,i] for i in 1:size(vd,2)];

# indexes of monomials obtained after multiplication by x[1] and x[2]

Cidx1 = indexin(idx1,vdv);
Cidx2 = indexin(idx2,vdv);
Cidx3 = indexin(idx3,vdv);

C1 = Md[:,sort(Cidx1)];
C2 = Md[:,sort(Cidx2)];
C3 = Md[:,sort(Cidx3)];

Ax1bar = C \ C1;
Ax2bar = C \ C2;
Ax3bar = C \ C3;

# X1 and X2 are the matrices encoding the multiplication operators

X1 = G * Ax1bar * inv(G);
X2 = G * Ax2bar * inv(G);
X3 = G * Ax3bar * inv(G);

X1=(X1+X1')/2;
X2=(X2+X2')/2;
X3=(X3+X3')/2;

# Now we extract the atoms of the Dirac measure

rn = rand(n);
rn /= norm(rn);
A = rn[1]*X1+rn[2]*X2 + rn[3]*X3;

T, Q, D = schur(Symmetric(A));

# each element of xsol is an extracted solution
xsol = map(j -> [Q[j,:]'*X1*Q[:,j], Q[j,:]'*X2*Q[:,j], Q[j,:]'*X3*Q[:,j]]  , 1:r);

return rn,xsol
end

extract (generic function with 1 method)

In [10]:
opt,sol,data = tssos_first(poly_mpo, variables(poly_mpo), order=1, TS=false);
Md=data.moment[1];
print(Md)
vd=data.basis;
print(vd)
n=3;
r = 1;
s = 1;

xsol=extract(Md,vd,s,n);
print(xsol)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[4]
[1]
-----------------------------------------------------------------------------
Assembling the SDP...
There are 10 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 1 (scalarized: 10)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
El

In [11]:
xsol[2][]

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [13]:
poly_mpo([0,0,0])

0.0